<a href="https://colab.research.google.com/github/asmita-mukherjee/LearningStuff/blob/main/Causal_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get the Data from kaggle

In [ ]:
!pip install -q opendatasets

In [ ]:
import opendatasets as od

In [ ]:
# download custom tokenizer and vocab
od.download("https://www.kaggle.com/datasets/asmitamukh/tinystories-custom-bpe")

#download the train and validation dataset
od.download("https://www.kaggle.com/datasets/thedevastator/tinystories-narrative-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: asmitamukh
Your Kaggle Key: ··········


100%|██████████| 292k/292k [00:00<00:00, 877kB/s]


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

 asmitamukh
Your Kaggle Key: ··········


100%|██████████| 576M/576M [00:21<00:00, 28.5MB/s]


In [ ]:
import logging
logging.basicConfig(format="%(asctime)s - %(message)s",level=logging.INFO,force=True)

In [ ]:
!pip install -q torchinfo wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
from torchinfo import summary

In [ ]:
import torch

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import torch
import torch.nn as nn

In [ ]:
import numpy as np

In [ ]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import wandb
wandb.login(key="b9fca33ebeeea7e66c03c7145a1072cc1bb07412")

2024-01-11 15:01:29,826 - NumExpr defaulting to 2 threads.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
BATCH_SIZE=2

In [ ]:
torch.set_default_dtype(torch.float16)

## helper functions

In [ ]:
# Read the vocab and merge rules

with open("/content/tinystories-custom-bpe/merge_rules.pkl","rb") as f:
    merge_rules = pickle.load(f)

with open("/content/tinystories-custom-bpe/vocab.pkl","rb") as f:
    vocab = pickle.load(f)

In [ ]:
def tokenize(text,merge_rules=merge_rules):
    '''Return tokens from the text'''
    text = text.lower() # our vocabulary is uncased.
    words = nltk.word_tokenize(text)

    ##preparing dict where word is the key and the tokens is its value
    word_dict = dict()
    for word in words:
        word_dict[word] = []
        for char in word:
            word_dict[word].append(char)

    for merge_rule in merge_rules.keys():
        #go through each merge rule and tokenize each word from left to right
        for word in word_dict.keys():
            tokens = word_dict[word]
            idx=0
            while(idx<len(tokens)-1):
                pair = (tokens[idx],tokens[idx+1])
                if pair==merge_rule:
                    #merge the token of the word as per the merge rule
                    #if merged we dont increase the index of the tokens because the next pair should include the new pair and the next char
                    tokens = tokens[:idx]+[tokens[idx]+tokens[idx+1]]+tokens[idx+2:]
                    word_dict[word] = tokens
                else:
                    #if not merged then we slide the window over
                    idx = idx+1

    return word_dict

## Load training and testing dataset

In [ ]:
train_data = pd.read_csv("/content/tinystories-narrative-classification/train.csv",nrows=50)
test_data = pd.read_csv("/content/tinystories-narrative-classification/validation.csv",nrows=5)
train_data = train_data.fillna("")
test_data = test_data.fillna("")

In [ ]:
len(train_data),len(test_data)

(50, 5)

In [ ]:
min(len(text.split()) for text in train_data.text),max(len(text.split()) for text in train_data.text)

(80, 201)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50 non-null     object
dtypes: object(1)
memory usage: 528.0+ bytes


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5 non-null      object
dtypes: object(1)
memory usage: 168.0+ bytes


### Causal Lm data preparation

In [ ]:
val = tokenize("I have an apple").values()
[i for item in val for i in item ]

['i', 'have', 'an', 'apple']

In [ ]:
def create_train_dataset_causal_lm(texts,context_window=64):
    dataset = {"input":[],"labels":[]}

    for text in tqdm(texts):
        tokens = tokenize(text)
        tokens = [token for word_tokens in tokens.values() for token in word_tokens]

        if len(tokens)<context_window:
          dataset["input"].append(' '.join(tokens))
          dataset["labels"].append("<eos>")
          continue

        for idx in range(len(tokens)-context_window):
            dataset["input"].append(' '.join(tokens[idx:idx+context_window]))

            if (idx+context_window)<len(tokens):

              dataset["labels"].append(tokens[idx+context_window]) #the label window shifts by one
              #logging.info("context larger than token length")

            else:
              dataset["labels"] = "<eos>"

    return pd.DataFrame(dataset)



In [ ]:
create_train_dataset_causal_lm(["I have an apple"])

  0%|          | 0/1 [00:00<?, ?it/s]

,input,labels
0,i have an apple,<eos>


In [ ]:
def sen_to_tensor(sen,vocab=vocab,merge_rules=merge_rules):
    '''Each sen will be converted to a vector where each token will be mapped to its index in vocab'''
    sen = sen.lower()
    token_dict = tokenize(sen,merge_rules)
    words = nltk.word_tokenize(sen)
    vocab = list(vocab)
    list_of_tokens = []
    for word in words:
        list_of_tokens.extend(token_dict[word])

    #find the index of the token in the vocab
    tensor = np.zeros(len(vocab),dtype=np.float16)
    for idx,token in enumerate(list_of_tokens):
        tensor[idx] = vocab.index(token)
    return tensor

In [ ]:
sen_to_tensor("I have an apple")

array([2568., 6584., 9776., ...,    0.,    0.,    0.], dtype=float16)

In [ ]:
def batch_dataset(inputs:list[str],labels:list[str],batch_size:int):
    '''
        takes list of input and labels and batches them
    '''
    input_batched = []
    label_batched = []

    for batch,idx in enumerate(range(0,len(inputs),batch_size)):
        input_batched.append(inputs[idx:idx+batch_size])
        label_batched.append(labels[idx:idx+batch_size])

    return input_batched,label_batched


In [ ]:
train_dataset = create_train_dataset_causal_lm(train_data.text)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
train_dataset

,input,labels
0,"one day , a little girl named lily found needl...",after
1,"day , a little girl named lily found needle in...",finished
2,", a little girl named lily found needle in her...",thanked
3,a little girl named lily found needle in her r...,fixing
4,little girl named lily found needle in her roo...,both
...,...,...
715,tim . loved ice-cream so much one day he saw a...,walked
716,. loved ice-cream so much one day he saw an an...,away
717,loved ice-cream so much one day he saw an anci...,get
718,ice-cream so much one day he saw an ancient tr...,enjoy


In [ ]:
test_dataset = create_train_dataset_causal_lm(test_data.text)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
len(train_dataset),len(test_dataset)

(720, 72)

> Save the dataset

In [ ]:
input_batch,label_batch = batch_dataset(train_dataset["input"],train_dataset["labels"],batch_size=BATCH_SIZE)

In [ ]:
input_batch_test,label_batch_test = batch_dataset(test_dataset["input"],test_dataset["labels"],batch_size=BATCH_SIZE)

In [ ]:
input_batch_test[0]

0    spot . saw the shiny car and said , `` wow kit...
1    once upon a time , in big forest there lived r...
Name: input, dtype: object

Expected shape of input tensor nos_of_batches,batch_size,vocab_size

converting input batch to tensor

In [ ]:
vocab.add("<eos>")

In [ ]:
"<eos>" in vocab

True

In [ ]:
list(vocab).index("<eos>")

628

In [ ]:
def convert_data_batch_to_tensor(input_batch,batch_size):

    nos_of_batches = len(input_batch)
    batch_size = batch_size
    vocab_size = len(vocab)
    input_tensor_batch = np.zeros((nos_of_batches,batch_size,vocab_size),dtype=np.float16)
    for batch,inputs in tqdm(enumerate(input_batch)):
        for elm_nos,text in enumerate(inputs):
            input_tensor_batch[batch][elm_nos]= sen_to_tensor(text)
    return input_tensor_batch


In [ ]:
def convert_label_batch_to_tensor(label_batch,batch_size):
  nos_of_batches = len(label_batch)
  vocab_list = list(vocab)

  for batch in label_batch:
    output_tensor_batch = np.zeros((nos_of_batches,batch_size),dtype=np.long)

    for batch,labels in tqdm(enumerate(label_batch)):
        for elm_nos,label in enumerate(labels):
          output_tensor_batch[batch][elm_nos]= vocab_list.index(label)
    return output_tensor_batch


In [ ]:
input_train_batch = convert_data_batch_to_tensor(input_batch,batch_size=BATCH_SIZE)
input_test_batch = convert_data_batch_to_tensor(input_batch_test,batch_size=BATCH_SIZE)

input_train_batch.shape,input_test_batch.shape

0it [00:00, ?it/s]

0it [00:00, ?it/s]

((360, 2, 9841), (36, 2, 9841))

Converting output batch to tensor

In [ ]:


label_train_batch =  convert_label_batch_to_tensor(label_batch,batch_size=BATCH_SIZE)
label_test_batch =  convert_label_batch_to_tensor(label_batch_test,batch_size=BATCH_SIZE)


label_train_batch.shape,label_test_batch.shape

<ipython-input-39-6a2397d10cdd>:6: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  output_tensor_batch = np.zeros((nos_of_batches,batch_size),dtype=np.long)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

((360, 2), (36, 2))

In [ ]:
class BareLLM(nn.Module):
    def __init__(self,vocab_size,embed_dim,hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embed_dim)
        self.layer_norm = nn.LayerNorm(embed_dim)
        self.flatten = nn.Flatten()
        self.Linear1 = nn.Linear(vocab_size*embed_dim,hidden_dim)
        self.relu1 = nn.ReLU()
        self.Linear2 = nn.Linear(hidden_dim,vocab_size)
        self.relu2 = nn.ReLU()

    def forward(self,input_tensor):
        logging.debug(f"Shape of input tensor {input_tensor.shape}")
        embedded_tensor = self.embedding(input_tensor)
        logging.debug(f"Shape of embedding matrix {embedded_tensor.shape}")
        norm_tensor = self.layer_norm(embedded_tensor)
        logging.debug(f"Shape of matrix after norm {norm_tensor.shape}")
        flattened_tensor = self.flatten(norm_tensor)
        logging.debug(f"Shape of flattened tensor {flattened_tensor.shape}")
        linear_output_1 = self.Linear1(flattened_tensor)
        logging.debug(f"Shape of tensor after linear layer 1 {linear_output_1.shape}")
        relu_output_1 = self.relu1(linear_output_1)
        linear_output_2 = self.Linear2(relu_output_1)
        logging.debug(f"Shape of tensor after linear layer 2 {linear_output_2.shape}")
        relu_output_2 = self.relu2(linear_output_2)
        return relu_output_2


In [ ]:
model = BareLLM(vocab_size=len(vocab),embed_dim=768,hidden_dim=128)

In [ ]:
summary(model)

Layer (type:depth-idx)                   Param #
BareLLM                                  --
├─Embedding: 1-1                         7,557,888
├─LayerNorm: 1-2                         1,536
├─Flatten: 1-3                           --
├─Linear: 1-4                            967,409,792
├─ReLU: 1-5                              --
├─Linear: 1-6                            1,269,489
├─ReLU: 1-7                              --
Total params: 976,238,705
Trainable params: 976,238,705
Non-trainable params: 0

In [ ]:
torch.cuda.empty_cache()

In [ ]:
print(model.parameters())

<generator object Module.parameters at 0x7bd6283bf300>


In [ ]:
model = model.to(device)

> Checking forward pass for one iteration

In [ ]:
s_output = model(torch.tensor(input_train_batch[0],dtype=torch.long,device=device))

In [ ]:
print(s_output.shape)

torch.Size([2, 9841])


### Write Training Loop

In [ ]:
torch.cuda.empty_cache()

In [ ]:
vocab_size = len(vocab)
vocab_size

9841

In [ ]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.0001,momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

def train_one_epoch(input_batch,label_batch):
    running_loss = 0 #captures the loss at each iteration of the dataset
    last_loss = 0 #captures the loss at the training of the epoch

    for batch_nos,(inputs,labels) in enumerate(zip(input_batch,label_batch)):
        logging.debug(f"Batch nos {batch_nos}")
        #at the start of every batch zero grad optimizer to clean the gradients
        optimizer.zero_grad()

        #output for the batch
        logits = model(inputs)

        #compute loss
        loss = loss_fn(logits,labels)
        loss.backward()

        #apply the updates to all parameters
        optimizer.step()

        running_loss += loss.item()
        if batch_nos%100 ==99:
            last_loss = running_loss/500
            logging.info(f"For batch {batch_nos+1} loss at 500 samples is {last_loss}")
            running_loss = 0

    return last_loss


In [ ]:
def infer_from_logits(logits):
    vocab_index = torch.argmax(logits,dim=1)
    logging.debug(f"sha")
    vocab_index = vocab_index.cpu().detach().numpy()
    vocab_list = list(vocab)
    list_of_outputs = [vocab_list[idx] for idx in vocab_index]
    return list_of_outputs



In [ ]:
def display_output(list_of_inputs,list_of_pred_values,list_of_labels):
    for input,pred,label in zip(list_of_inputs,list_of_pred_values,list_of_labels):
        print("Actual -->")
        print(f"Input Text->{input} ***Actual Label***->{label}")
        print("========================================")
        print("Predicted -->")
        print(f"Input Text->{input} ***Predicted Label***->{pred}")
        print("========================================\n")



In [ ]:
wandb.init(project="bare_min_llm_8th_jan",name="correct_causal_lm_8th_jan")
wandb.watch(model, log_freq=100)

wandb: Currently logged in as: asmitaxyz. Use `wandb login --relogin` to force relogin


[]

In [ ]:
EPOCH = 10

for epoch in range(EPOCH):
    logging.info(f"Starting training at {epoch+1}")
    model.train(True) #making sure gradient tracking is on
    avg_loss = train_one_epoch(torch.tensor(input_train_batch,dtype=torch.long,device=device),
                               torch.tensor(label_train_batch,dtype=torch.long,device=device))

    running_val_loss = 0.0

    model.eval() #set the model to eval model(handels dropout calc etc)
    with torch.no_grad():
        for i,(v_input,v_label,v_input_text,v_label_text) in enumerate(zip(input_test_batch,label_test_batch,input_batch_test,label_batch_test)):
            v_input = torch.tensor(v_input,dtype=torch.long,device=device)
            v_label = torch.tensor(v_label,dtype=torch.long,device=device)
            v_output = model(v_input)
            predicted_outputs = infer_from_logits(v_output)
            v_loss = loss_fn(v_output,v_label)
            running_val_loss += v_loss
            if i==0:
                display_output(v_input_text,predicted_outputs,v_label_text)
            if i==len(input_test_batch)-1:
                display_output(v_input_text,predicted_outputs,v_label_text)

    avg_vloss = running_val_loss / (epoch + 1)
    wandb.log({"train_loss":avg_loss})
    wandb.log({"val_loss":avg_vloss})
    wandb.log({"epoch":epoch})
    logging.info('LOSS train {} valid {}'.format(avg_loss, avg_vloss))





2024-01-11 15:03:51,890 - Starting training at 1
2024-01-11 15:04:25,828 - For batch 100 loss at 500 samples is 1.89353125
2024-01-11 15:04:57,777 - For batch 200 loss at 500 samples is 1.83959375
2024-01-11 15:05:36,576 - For batch 300 loss at 500 samples is 1.83990625


Actual -->
Input Text->spot . saw the shiny car and said , `` wow kitty your is so bright clean ! '' smiled replied thank you i polish it every day after playing with felt thirsty they found a small pond clear water drank very happy played together all became best friends ***Actual Label***-><eos>
Predicted -->
Input Text->spot . saw the shiny car and said , `` wow kitty your is so bright clean ! '' smiled replied thank you i polish it every day after playing with felt thirsty they found a small pond clear water drank very happy played together all became best friends ***Predicted Label***->not

Actual -->
Input Text->once upon a time , in big forest there lived rhinoceros named ro x y . loved to climb she climbed trees rocks and hills one day found an icy hill had never seen anything like it before was shiny cold wanted tried the but very slippery again kept falling down sad so much then saw little bird billy that asked `` why are ***Actual Label***->you
Predicted -->
Input Text->once

2024-01-11 15:05:42,862 - LOSS train 1.83990625 valid 331.25
2024-01-11 15:05:42,865 - Starting training at 2


Actual -->
Input Text->once upon a time , there was kind farmer . he had big cow the sad did not know why one day little boy came to farm saw kneeled down talk `` are you ? '' asked said i am lonely want friend heard wanted help so got another be friends with happy now they played together every and two cows all lived happily ***Actual Label***->ever
Predicted -->
Input Text->once upon a time , there was kind farmer . he had big cow the sad did not know why one day little boy came to farm saw kneeled down talk `` are you ? '' asked said i am lonely want friend heard wanted help so got another be friends with happy now they played together every and two cows all lived happily ***Predicted Label***->not

Actual -->
Input Text->upon a time , there was kind farmer . he had big cow the sad did not know why one day little boy came to farm saw kneeled down talk `` are you ? '' asked said i am lonely want friend heard wanted help so got another be friends with happy now they played together ev

2024-01-11 15:06:20,633 - For batch 100 loss at 500 samples is 1.838921875
2024-01-11 15:06:52,030 - For batch 200 loss at 500 samples is 1.8394375
2024-01-11 15:07:23,351 - For batch 300 loss at 500 samples is 1.839734375


...

Actual -->
Input Text->once upon a time , there was kind farmer . he had big cow the sad did not know why one day little boy came to farm saw kneeled down talk `` are you ? '' asked said i am lonely want friend heard wanted help so got another be friends with happy now they played together every and two cows all lived happily ***Actual Label***->ever
Predicted -->
Input Text->once upon a time , there was kind farmer . he had big cow the sad did not know why one day little boy came to farm saw kneeled down talk `` are you ? '' asked said i am lonely want friend heard wanted help so got another be friends with happy now they played together every and two cows all lived happily ***Predicted Label***->now

Actual -->
Input Text->upon a time , there was kind farmer . he had big cow the sad did not know why one day little boy came to farm saw kneeled down talk `` are you ? '' asked said i am lonely want friend heard wanted help so got another be friends with happy now they played together ev

2024-01-11 15:21:33,746 - For batch 100 loss at 500 samples is 1.838
2024-01-11 15:22:05,656 - For batch 200 loss at 500 samples is 1.838578125
2024-01-11 15:22:37,199 - For batch 300 loss at 500 samples is 1.838953125


Actual -->
Input Text->spot . saw the shiny car and said , `` wow kitty your is so bright clean ! '' smiled replied thank you i polish it every day after playing with felt thirsty they found a small pond clear water drank very happy played together all became best friends ***Actual Label***-><eos>
Predicted -->
Input Text->spot . saw the shiny car and said , `` wow kitty your is so bright clean ! '' smiled replied thank you i polish it every day after playing with felt thirsty they found a small pond clear water drank very happy played together all became best friends ***Predicted Label***->now

Actual -->
Input Text->once upon a time , in big forest there lived rhinoceros named ro x y . loved to climb she climbed trees rocks and hills one day found an icy hill had never seen anything like it before was shiny cold wanted tried the but very slippery again kept falling down sad so much then saw little bird billy that asked `` why are ***Actual Label***->you
Predicted -->
Input Text->once

2024-01-11 15:23:04,093 - LOSS train 1.838953125 valid 33.09375


Actual -->
Input Text->once upon a time , there was kind farmer . he had big cow the sad did not know why one day little boy came to farm saw kneeled down talk `` are you ? '' asked said i am lonely want friend heard wanted help so got another be friends with happy now they played together every and two cows all lived happily ***Actual Label***->ever
Predicted -->
Input Text->once upon a time , there was kind farmer . he had big cow the sad did not know why one day little boy came to farm saw kneeled down talk `` are you ? '' asked said i am lonely want friend heard wanted help so got another be friends with happy now they played together every and two cows all lived happily ***Predicted Label***->now

Actual -->
Input Text->upon a time , there was kind farmer . he had big cow the sad did not know why one day little boy came to farm saw kneeled down talk `` are you ? '' asked said i am lonely want friend heard wanted help so got another be friends with happy now they played together ev

Testing

In [ ]:
@torch.no_grad()
def init_weights(m):
  print(m)
  if type(m) == nn.Linear:
    m.weight.fill_(1.0)
    print(m.weight)

net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
net.apply(init_weights)


Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

In [ ]:
!pip install transformers

In [ ]:
from transformers import GPT2LMHeadModel
model_hf = GPT2LMHeadModel.from_pretrained("gpt2")
sd_hf = model_hf.state_dict()
sd_hf

Next study : https://github.com/karpathy/minGPT/blob/master/projects/adder/adder.py

How do they batvh  the data , how do they train the model